In [ ]:
# Importieren der erforderlichen Bibliotheken
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

In [ ]:
# Daten laden
ratings_df = pd.read_csv("ml-32m/ratings.csv")
movies_df = pd.read_csv("ml-32m/movies.csv")

In [ ]:
# Verbinden der DataFrames basierend auf der movieId
ratings = pd.merge(ratings_df, movies_df, on='movieId', how='inner')

In [ ]:
# Daten untersuchen
print("Schema:")
print(ratings.info())

In [ ]:
print("Anzahl der Zeilen und Spalten:")
print(f"Zeilen: {ratings.shape[0]}, Spalten: {ratings.shape[1]}")

In [ ]:
print("Erste Zeilen:")
print(ratings.head())

In [ ]:
# Temporäre Änderung der Anzeigeoptionen für float-Werte
with pd.option_context('display.float_format', '{:,.2f}'.format):
    print("Descriptive Stats:")
    print(ratings['rating'].describe())

In [ ]:
# Anzahl Ratings insgesamt
total_ratings = ratings.shape[0]
print(f"Anzahl Ratings insgesamt: {total_ratings}")

# Anzahl verschiedener Nutzer
unique_users = ratings['userId'].nunique()
print(f"Anzahl verschiedener Nutzer: {unique_users}")

# Anzahl verschiedener Filme
unique_movies = ratings['movieId'].nunique()
print(f"Anzahl verschiedener Filme: {unique_movies}")

# Anzahl verschiedener Genres
# Genres aus dem 'genres'-Feld extrahieren
all_genres = movies_df['genres'].str.split('|').explode().unique()
unique_genres = len(all_genres)
print(f"Anzahl verschiedener Genres: {unique_genres}")

In [ ]:
# Doppelte Kombinationen von userId und movieId finden
duplicates = ratings.groupby(['userId', 'movieId']).size()
duplicate_count = (duplicates > 1).sum()
print(f"Anzahl der doppelten Bewertungen: {duplicate_count}")

In [ ]:
# Bewertungen pro Nutzer berechnen
user_ratings_count = ratings.groupby('userId').size()

# Nutzer mit mindestens 1, 3 und 5 Bewertungen zählen
users_1 = (user_ratings_count == 1).sum()
users_min_1 = (user_ratings_count >= 1).sum()
users_min_30 = (user_ratings_count >= 30).sum()
users_min_50 = (user_ratings_count >= 50).sum()

print(f"Anzahl der Nutzer mit genau 1 Bewertung: {users_1}")
print(f"Anzahl der Nutzer mit mindestens 1 Bewertung: {users_min_1}")
print(f"Anzahl der Nutzer mit mindestens 30 Bewertungen: {users_min_30}")
print(f"Anzahl der Nutzer mit mindestens 50 Bewertungen: {users_min_50}")

In [ ]:
# Nutzer mit ausschließlich demselben Rating
user_unique_ratings = ratings.groupby('userId')['rating'].nunique()
num_users_with_same_ratings = (user_unique_ratings == 1).sum()
print(f"Anzahl der Benutzer mit ausschließlich demselben Rating: {num_users_with_same_ratings}")

In [ ]:
# Bewertungen pro Film berechnen
movie_ratings_count = ratings.groupby('movieId').size()

# Filme mit mindestens 1, 3 und 5 Bewertungen zählen
movies_min_0 = (movie_ratings_count == 0).sum()
movies_min_1 = (movie_ratings_count >= 1).sum()
movies_min_3 = (movie_ratings_count >= 3).sum()
movies_min_5 = (movie_ratings_count >= 5).sum()

print(f"Anzahl der Filme mit mindestens 0 Bewertung: {movies_min_0}")
print(f"Anzahl der Filme mit mindestens 1 Bewertung: {movies_min_1}")
print(f"Anzahl der Filme mit mindestens 3 Bewertungen: {movies_min_3}")
print(f"Anzahl der Filme mit mindestens 5 Bewertungen: {movies_min_5}")

In [ ]:
# Durchschnittliche Bewertung pro Film
avg_rating_per_movie = ratings.groupby('movieId')['rating'].mean()
print(avg_rating_per_movie.describe())

In [ ]:
# Durchschnittliche Bewertung pro Benutzer
avg_rating_per_user = ratings.groupby('userId')['rating'].mean()
print(avg_rating_per_user.describe())

In [ ]:
# Ratings-Verteilung
rating_distribution = ratings['rating'].value_counts().sort_index()

# Visualisierungen
plt.figure(figsize=(10, 6))
rating_distribution.plot(kind='bar', color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Anzahl der Bewertungen')

# Ticks für die Y-Achse festlegen (lineare Skala)
ticks = [1_000_000, 2_000_000, 3_000_000, 4_000_000, 5_000_000, 6_000_000, 7_000_000, 8_000_000]
plt.yticks(ticks)  # Manuelle Ticks setzen

# Y-Achse formatieren: Keine wissenschaftliche Notation, sondern Klartext
def linear_format(x, _):
    return f'{int(x):,}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(linear_format))

# Achsenbeschriftungen anpassen
plt.xticks(rotation=0)  # X-Achse bleibt aufrecht

# Diagramm speichern
plt.savefig("RatingsVerteilung_MovieLens32M.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Verteilung der Bewertungen pro Nutzer
user_ratings_count.plot.hist(bins=50, color='lightgreen')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Nutzer')
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))
plt.savefig("AnzahlBewertungenNutzer_MovieLens32M.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Top 10 Nutzer mit den meisten Bewertungen
top_users = user_ratings_count.nlargest(10)
top_users.plot(kind='bar', color='orange', title='Top 10 Nutzer mit den meisten Bewertungen')
plt.xlabel('Nutzer-ID')
plt.ylabel('Anzahl der Bewertungen')
plt.show()

In [ ]:
# Histogramm der durchschnittlichen Bewertungen pro Benutzer
plt.figure(figsize=(10, 6))
sns.histplot(avg_rating_per_user, bins=20, color='purple')
plt.title('Durchschnittliche Bewertung pro Benutzer')
plt.xlabel('Durchschnittliche Bewertung')
plt.ylabel('Anzahl der Benutzer')
plt.show()

In [ ]:
# Verteilung der Anzahl von Bewertungen pro Film
movie_rating_counts = ratings.groupby('movieId').size()

plt.figure(figsize=(10, 6))
plt.hist(movie_rating_counts, bins=50, color='lightgreen')
plt.title('Verteilung der Anzahl von Bewertungen pro Film')
plt.xlabel('Anzahl der Bewertungen')
plt.ylabel('Anzahl der Filme')

# Achse auf logarithmische Skala setzen
plt.yscale('log')

# Y-Achse mit normalen Zahlen beschriften
def log_format(x, _):
    return f'{int(x):,}' if x >= 1 else ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(log_format))

plt.show()

In [ ]:
# Durchschnittliche Bewertung der Top 10 Filme (nach Anzahl Bewertungen)
top_movies_counts = ratings['movieId'].value_counts().head(10)
top_movie_titles = movies_df[movies_df['movieId'].isin(top_movies_counts.index)]
movie_id_to_title = dict(zip(top_movie_titles['movieId'], top_movie_titles['title']))
top_avg_ratings = avg_rating_per_movie.loc[top_movies_counts.index]
top_avg_ratings.index = top_avg_ratings.index.map(movie_id_to_title)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_avg_ratings.index, y=top_avg_ratings.values, palette='viridis')
plt.title('Durchschnittliche Bewertung der Top 10 Filme')
plt.xlabel('Buchtitel')
plt.ylabel('Durchschnittliche Bewertung')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Top 10 Filme mit den meisten Bewertungen
top_movies_counts = ratings['movieId'].value_counts().head(10)
top_movie_titles = movies_df[movies_df['movieId'].isin(top_movies_counts.index)]
movie_id_to_title = dict(zip(top_movie_titles['movieId'], top_movie_titles['title']))
top_movies_counts.index = top_movies_counts.index.map(movie_id_to_title)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_movies_counts.index, y=top_movies_counts.values, palette='viridis')
plt.title('Top 10 Filme mit den meisten Bewertungen')
plt.xlabel('Filmtitel')
plt.ylabel('Anzahl der Bewertungen')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Durchschnittliche Bewertung pro Jahrzehnt
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)
ratings_with_year = pd.merge(ratings, movies_df[['movieId', 'year']], on='movieId')
ratings_with_year['decade'] = (ratings_with_year['year'] // 10) * 10
avg_rating_per_decade = ratings_with_year.groupby('decade')['rating'].mean()
avg_rating_per_decade.plot(kind='line', marker='o', title='Durchschnittliche Bewertung pro Jahrzehnt')
plt.xlabel('Dekade')
plt.ylabel('Durchschnittliche Bewertung')
plt.show()